In [2]:
import torch
from torch import nn
import numpy as np
import mdshare 
import mdtraj as md 
import glob
import torch.nn.functional as F
from ase import Atoms
from nglview import show_ase
import nglview as ng
import matplotlib.pyplot as plt

In [3]:
# define a CGAE

class cgae(nn.Module):
    
    def __init__(self, n_atoms, n_cgs):
        super().__init__()

        assign_map = torch.randn(n_atoms, n_cgs)
        decode = torch.randn(n_cgs, n_atoms)
        
        
        self.n_atoms = n_atoms 
        self.assign_map = nn.Parameter(assign_map)
        self.decode = nn.Parameter(decode)
        
    def forward(self, xyz, tau=1.0):
        
        # recenter coordinates 
        xyz = xyz.reshape(-1, self.n_atoms, 3)
        shift = xyz.mean(1)
        xyz = xyz - shift.unsqueeze(1)
        
        # get discrete assignment map
        M = F.gumbel_softmax(self.assign_map, dim=-1)
        M_norm = M / M.sum(-2).unsqueeze(-2)
        
        cg_xyz = torch.einsum('bij,in->bnj', xyz, M_norm)
        xyz_recon = torch.einsum('bnj,ni->bij', cg_xyz, self.decode)
        
        return xyz, xyz_recon, M, M_norm, cg_xyz

In [4]:
ae_L6 = torch.load('LANGEVIN/Models/ae_CG6.pth').to('cpu')
ae_L6.eval()

ae_NH6 = torch.load('NOSE_HOOVER/Models/ae_CG6.pth').to('cpu')
ae_NH6.eval()

ae_NH62 = torch.load('NOSE_HOOVER_2/Models/ae_CG6.pth').to('cpu')
ae_NH62.eval()

ae_L24 = torch.load('LANGEVIN/Models/ae_CG24.pth').to('cpu')
ae_L24.eval()

ae_NH24 = torch.load('NOSE_HOOVER/Models/ae_CG24.pth').to('cpu')
ae_NH24.eval()

ae_NH224 = torch.load('NOSE_HOOVER_2/Models/ae_CG24.pth').to('cpu')
ae_NH224.eval()

cgae()

In [5]:
# build dataset and dataloader 
k = 10000
tau = 0.4

data_L = np.load('LANGEVIN/N_70.npy')
data_NH = np.load('NOSE_HOOVER/N_70.npy')
data_NH2 = np.load('NOSE_HOOVER_2/N_70.npy')


test_xyz_L  = torch.Tensor(data_L[k:])
test_xyz_NH  = torch.Tensor(data_NH[k:])
test_xyz_NH2  = torch.Tensor(data_NH2[k:])


In [6]:
xyz_L6, xyz_recon_L6, M_L6, M_norm_L6, cg_xyz_L6 = ae_L6(test_xyz_L[50], tau)
xyz_NH6, xyz_recon_NH6, M_NH6, M_norm_NH6, cg_xyz_NH6 = ae_NH6(test_xyz_NH[0], tau)
xyz_NH26, xyz_recon_NH26, M_NH26, M_norm_NH26, cg_xyz_NH26 = ae_NH6(test_xyz_NH2[0], tau)

xyz_L24, xyz_recon_L24, M_L24, M_norm_L24, cg_xyz_L24 = ae_L24(test_xyz_L[50], tau)
xyz_NH24, xyz_recon_NH24, M_NH24, M_norm_NH24, cg_xyz_NH24 = ae_NH24(test_xyz_NH[0], tau)
xyz_NH224, xyz_recon_NH224, M_NH224, M_norm_NH224, cg_xyz_NH224 = ae_NH224(test_xyz_NH2[0], tau)

In [7]:
import numpy as np
import os

In [2]:
data_L = np.load('LANGEVIN/N_70.npy')
data_NH = np.load('NOSE_HOOVER/N_70.npy')
data_NH2 = np.load('NOSE_HOOVER_2/N_70.npy')

In [21]:
xyz = xyz_L6.reshape(70, 3).detach().cpu().numpy()
f = open('originalPR.txt', 'x')
f.write(str(70)+'\n')
for i in np.arange(70):
    if i==69:
        f.write(str(i)+' '+'C'+' '+str(xyz[i, 0])+' '+str(xyz[i, 1])+' '+str(xyz[i, 2]))
    else:
        f.write(str(i)+' '+'C'+' '+str(xyz[i, 0])+' '+str(xyz[i, 1])+' '+str(xyz[i, 2])+'\n')
f.close()    

In [22]:
xyz = cg_xyz_L6.reshape(6, 3).detach().cpu().numpy()
f = open('CG6PR.txt', 'x')
f.write(str(6)+'\n')
for i in np.arange(6):
    if i==69:
        f.write(str(i)+' '+'C'+' '+str(xyz[i, 0])+' '+str(xyz[i, 1])+' '+str(xyz[i, 2]))
    else:
        f.write(str(i)+' '+'C'+' '+str(xyz[i, 0])+' '+str(xyz[i, 1])+' '+str(xyz[i, 2])+'\n')
f.close()

In [23]:
xyz = cg_xyz_L24.reshape(24, 3).detach().cpu().numpy()
f = open('CG24PR.txt', 'x')
f.write(str(24)+'\n')
for i in np.arange(24):
    if i==69:
        f.write(str(i)+' '+'C'+' '+str(xyz[i, 0])+' '+str(xyz[i, 1])+' '+str(xyz[i, 2]))
    else:
        f.write(str(i)+' '+'C'+' '+str(xyz[i, 0])+' '+str(xyz[i, 1])+' '+str(xyz[i, 2])+'\n')
f.close()

In [24]:
xyz = xyz_recon_L6.reshape(70, 3).detach().cpu().numpy()
f = open('recon6PR.txt', 'x')
f.write(str(70)+'\n')
for i in np.arange(70):
    if i==69:
        f.write(str(i)+' '+'C'+' '+str(xyz[i, 0])+' '+str(xyz[i, 1])+' '+str(xyz[i, 2]))
    else:
        f.write(str(i)+' '+'C'+' '+str(xyz[i, 0])+' '+str(xyz[i, 1])+' '+str(xyz[i, 2])+'\n')
f.close()    

In [25]:
xyz = xyz_recon_L24.reshape(70, 3).detach().cpu().numpy()
f = open('recon24PR.txt', 'x')
f.write(str(70)+'\n')
for i in np.arange(70):
    if i==69:
        f.write(str(i)+' '+'C'+' '+str(xyz[i, 0])+' '+str(xyz[i, 1])+' '+str(xyz[i, 2]))
    else:
        f.write(str(i)+' '+'C'+' '+str(xyz[i, 0])+' '+str(xyz[i, 1])+' '+str(xyz[i, 2])+'\n')
f.close()    

In [14]:
os.remove('polymer_ring.txt')

In [16]:
os.remove('polymer_ring_bond.txt')